In [2]:
def compute_ground_state(atom_string):
    import numpy as np
    from qiskit import Aer
    from qiskit.opflow import PauliSumOp
    from qiskit.utils import QuantumInstance
    from qiskit.algorithms import VQE
    from qiskit.algorithms.optimizers import COBYLA
    from qiskit.circuit.library import EfficientSU2
    from qiskit_nature.drivers import PySCFDriver
    from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
    from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper
    from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

    # 1. Set up the electronic structure problem
    driver = PySCFDriver(atom=atom_string, unit='Angstrom', basis='sto3g')
    problem = ElectronicStructureProblem(driver)

    # 2. Map the fermionic Hamiltonian to the qubit Hamiltonian
    mapper = BravyiKitaevMapper()
    converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)
    num_particles = (problem.molecule_data_transformed.num_alpha,
                    problem.molecule_data_transformed.num_beta)
    qubit_op = converter.convert(problem.second_q_ops()[0], num_particles=num_particles)

    # 3. Define the quantum instance, the ansatz, and the classical optimizer
    quantum_instance = QuantumInstance(Aer.get_backend('statevector_simulator'))
    ansatz = EfficientSU2(qubit_op.num_qubits, reps=1)
    optimizer = COBYLA(maxiter=1000)

    # 4. Run VQE
    vqe = VQE(ansatz, optimizer, quantum_instance=quantum_instance)
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return result.eigenvalue.real

# Example usage:
atom_config = 'Li .0 .0 .0'
energy = compute_ground_state(atom_config)
print(f"Ground state energy for {atom_config}: {energy:.4f}")


ModuleNotFoundError: No module named 'qiskit_nature'